In [2]:
!pip install meteostat
!pip install geopandas

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
PATH_TO_TRAIN = "train_dataset.csv"
PATH_TO_TEST = "test_dataset.csv"

In [7]:
cols_order =  ['id', 'area', '.geo',
               '2021-04-15','2021-04-16','2021-04-18','2021-04-19','2021-04-20','2021-04-22','2021-04-23','2021-04-25','2021-04-26','2021-04-27','2021-04-28','2021-04-29','2021-04-30','2021-05-01',
               '2021-05-02','2021-05-03','2021-05-04','2021-05-07','2021-05-08','2021-05-09','2021-05-10','2021-05-15','2021-05-16','2021-05-17','2021-05-19','2021-05-20','2021-05-21','2021-05-24',
               '2021-05-26','2021-05-27','2021-05-29','2021-06-02','2021-06-03','2021-06-04','2021-06-05','2021-06-06','2021-06-07','2021-06-09','2021-06-10','2021-06-12','2021-06-13','2021-06-16',
               '2021-06-18','2021-06-19','2021-06-20','2021-06-22','2021-06-25','2021-06-27','2021-06-28','2021-07-04','2021-07-05','2021-07-07','2021-07-08','2021-07-09','2021-07-13','2021-07-15',
               '2021-07-17','2021-07-20','2021-07-26','2021-07-27','2021-07-29','2021-07-31','2021-08-01','2021-08-07','2021-08-10','2021-08-11','2021-08-12','2021-08-13','2021-08-23','2021-08-27']
              
train_df = pd.read_csv(PATH_TO_TRAIN)
test_df = pd.read_csv(PATH_TO_TEST)

renamer = lambda x: x.split('mean_')[-1] if 'nd_mean' in x else x
train_df.rename(mapper = renamer, axis=1, inplace=True)
test_df.rename(mapper = renamer, axis=1, inplace=True)

              
train_df = train_df[cols_order + ['crop']]
test_df = test_df[cols_order]

train_df

,id,area,.geo,2021-04-15,2021-04-16,2021-04-18,2021-04-19,2021-04-20,2021-04-22,2021-04-23,...,2021-07-31,2021-08-01,2021-08-07,2021-08-10,2021-08-11,2021-08-12,2021-08-13,2021-08-23,2021-08-27,crop
0,3536,20,"{""type"":""Polygon"",""coordinates"":[[[42.49107269...",0.187150,0.072846,0.262300,0.261778,0.063134,0.062981,0.000000,...,0.000000,0.754859,0.000000,0.030883,0.000000,0.658016,0.126832,0.034858,0.614770,3
1,3739,45,"{""type"":""Polygon"",""coordinates"":[[[50.25545110...",0.245285,0.332928,0.349133,0.287182,0.064671,0.097138,0.024539,...,0.089721,0.706226,0.184231,0.062582,0.618475,0.601685,0.174914,0.000000,0.179612,4
2,1294,28,"{""type"":""Polygon"",""coordinates"":[[[37.99698998...",0.006391,0.013679,0.000000,0.007055,0.000000,0.120804,0.112471,...,0.411168,0.000000,0.357296,0.045370,0.172080,0.000000,0.090607,0.000000,0.054127,2
3,2859,19,"{""type"":""Polygon"",""coordinates"":[[[44.72154181...",0.293425,0.105976,0.318047,0.052500,0.153700,0.001917,0.045054,...,0.055983,0.152193,0.060656,0.352723,0.379196,0.327025,0.073492,0.061776,0.378900,5
4,3685,33,"{""type"":""Polygon"",""coordinates"":[[[34.77829934...",0.027983,0.130885,0.117227,0.063492,0.040565,0.024416,0.163886,...,0.611108,0.772962,0.000000,0.080592,0.000000,0.055220,0.157676,0.022251,0.012048,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,3772,74,"{""type"":""Polygon"",""coordinates"":[[[40.90052911...",0.066154,0.028770,0.000000,0.038282,0.000000,0.255329,0.095812,...,0.379482,0.700605,0.645532,0.000000,0.059090,0.631144,0.042483,0.118901,0.022464,6
4826,5191,109,"{""type"":""Polygon"",""coordinates"":[[[40.00883381...",0.243867,0.035939,0.013403,0.030722,0.246513,0.033869,0.000000,...,0.800815,0.634428,0.783913,0.093728,0.000000,0.776900,0.055721,0.057945,0.044182,6
4827,5226,58,"{""type"":""Polygon"",""coordinates"":[[[38.57789936...",0.090026,0.191255,0.019897,0.063581,0.000000,0.140111,0.143922,...,0.740523,0.781434,0.783490,0.006700,0.132444,0.000000,0.007225,0.669367,0.145505,6
4828,5390,14,"{""type"":""Polygon"",""coordinates"":[[[53.31082730...",0.301663,0.291399,0.295089,0.221428,0.326494,0.007247,0.006888,...,0.235802,0.000000,0.079623,0.049245,0.506127,0.528256,0.031836,0.000000,0.453151,5


In [9]:
import geopandas as gp


geo_coords = pd.read_csv('geo_coords.csv')[['id', 'centroid']]
geo_coords['x'] = gp.GeoSeries.from_wkt(f['centroid']).x
geo_coords['y'] = gp.GeoSeries.from_wkt(f['centroid']).y

train_df = train_df.merge(geo_coords, on = 'id', how = 'left')
test_df = test_df.merge(geo_coords, on = 'id', how = 'left')

train_df[['x', 'y']]

,x,y
0,51.413192,42.500831
1,56.605879,50.267528
2,53.836396,38.002610
3,55.935386,44.726868
4,52.907380,34.784405
...,...,...
4825,50.902175,40.906470
4826,51.804375,40.014197
4827,53.193000,38.585570
4828,55.183351,53.320243


In [2]:
def make_geo_cell(df):
    df['y_05'] = (df['y'] // 1.5).astype(int)
    df['x_05'] = (df['x'] // 1.5).astype(int)
    df['geo_cell'] = df['y_05'].astype(str) + '_' + df['x_05'].astype(str)
    df.drop(['y_05', 'x_05'], axis=1, inplace=True)
    
    
make_geo_cell(train_df)
make_geo_cell(test_df)

train_df['val'] = True
cols = ['crop','geo_cell', 'id', 'val']
df_pivot = train_df[cols].pivot_table(index = 'geo_cell', aggfunc = 'mean', values = 'val', columns = 'crop').fillna(0)
df_pivot.columns = [f'crop_{i}' for i in df_pivot.columns]
df_pivot.reset_index(inplace=True)
train_df.drop('val', axis=1, inplace=True)
df_pivot.head()

In [12]:
train_df = train_df.merge(df_pivot, on = 'geo_cell', how = 'left')
test_df = test_df.merge(df_pivot, on = 'geo_cell', how = 'left')

test_df.head()

,id,area,.geo,2021-04-15,2021-04-16,2021-04-18,2021-04-19,2021-04-20,2021-04-22,2021-04-23,...,x,y,geo_cell,crop_0,crop_1,crop_2,crop_3,crop_4,crop_5,crop_6
0,611,26,"{""type"":""Polygon"",""coordinates"":[[[35.96545926...",0.083038,0.196691,0.244827,0.000000,0.241219,0.259303,0.064947,...,52.384848,35.973536,23_34,True,True,True,True,True,0,True
1,6417,98,"{""type"":""Polygon"",""coordinates"":[[[39.50520518...",0.122470,0.091192,0.241003,0.000000,0.181600,0.125823,0.108175,...,52.065600,39.515554,26_34,True,True,True,True,True,True,True
2,3352,21,"{""type"":""Polygon"",""coordinates"":[[[39.30105409...",0.259347,0.000000,0.297489,0.033368,0.000000,0.285057,0.374026,...,49.945538,39.306785,26_33,True,True,True,True,True,True,True
3,4224,18,"{""type"":""Polygon"",""coordinates"":[[[33.08144648...",0.166209,0.111243,0.000000,0.201012,0.083885,0.000000,0.015033,...,52.550563,33.085696,22_35,True,True,True,True,True,0,0
4,3102,53,"{""type"":""Polygon"",""coordinates"":[[[45.10231339...",0.010862,0.000000,0.067479,0.234954,0.000000,0.000000,0.000000,...,55.372619,45.110255,30_36,True,True,True,True,True,True,True


In [23]:
X_array = train_df['x'].values
Y_array = train_df['y'].values


def near_coords(df):
    diff = lambda x : np.sum(abs(X_array - x) <= 0.05)
    df['x_n_near_05'] = df['x'].apply(f)

    diff = lambda x : np.sum(abs(X_array - x) <= 0.01)
    df['x_n_near_01'] = df['x'].apply(f)
    
    df['x_n_near_01_on_05'] = df['x_n_near_01']/df['x_n_near_05'] 

    diff = lambda x : np.sum(abs(Y_array - x) <= 0.01)
    df['y_n_near_01'] = df['y'].apply(f)

    diff = lambda x : np.sum(abs(Y_array - x) <= 0.05)
    df['y_n_near_05'] = df['y'].apply(f)
    
    df['y_n_near_01_on_05'] = df['y_n_near_01']/df['y_n_near_05'] 
    
near_coords(train_df)
near_coords(test_df)

In [17]:
date_cols = ['2021-04-15','2021-04-16','2021-04-18','2021-04-19','2021-04-20','2021-04-22','2021-04-23','2021-04-25','2021-04-26','2021-04-27','2021-04-28','2021-04-29','2021-04-30','2021-05-01',
               '2021-05-02','2021-05-03','2021-05-04','2021-05-07','2021-05-08','2021-05-09','2021-05-10','2021-05-15','2021-05-16','2021-05-17','2021-05-19','2021-05-20','2021-05-21','2021-05-24',
               '2021-05-26','2021-05-27','2021-05-29','2021-06-02','2021-06-03','2021-06-04','2021-06-05','2021-06-06','2021-06-07','2021-06-09','2021-06-10','2021-06-12','2021-06-13','2021-06-16',
               '2021-06-18','2021-06-19','2021-06-20','2021-06-22','2021-06-25','2021-06-27','2021-06-28','2021-07-04','2021-07-05','2021-07-07','2021-07-08','2021-07-09','2021-07-13','2021-07-15',
               '2021-07-17','2021-07-20','2021-07-26','2021-07-27','2021-07-29','2021-07-31','2021-08-01','2021-08-07','2021-08-10','2021-08-11','2021-08-12','2021-08-13','2021-08-23','2021-08-27']

agg_features = ["mean", "max", "var", "std", "argmax", "max_mean_diff", "argmin",
                "month_mean", "month_max", "month_var", "month_std", "month_argmax",
                "month_max_mean_diff", "month_argmin", "month_4", "month_5", "month_6", "month_7", "month_8"]

dates_4 = [col for col in date_cols if "-04-" in col]
dates_5 = [col for col in date_cols if "-05-" in col]
dates_6 = [col for col in date_cols if "-06-" in col]
dates_7 = [col for col in date_cols if "-07-" in col]
dates_8 = [col for col in date_cols if "-08-" in col]

def get_agg(df):
    df["mean"] = df[date_cols].values.mean(axis=1)
    df["min"] = df[date_cols].values.min(axis=1)
    df["max"] = df[date_cols].values.max(axis=1)
    df["var"] = df[date_cols].values.var(axis=1)
    df["std"] = df[date_cols].values.std(axis=1)
    df["argmax"] = np.argmax(df[date_cols].values, axis=1)
    df["argmin"] = np.argmin(df[date_cols].values, axis=1)
    df["max_mean_diff"] = df["max"] - df["mean"]
    
    month_sum = np.array([df[dates_4].values.sum(axis=1),
                          df[dates_5].values.sum(axis=1),
                          df[dates_6].values.sum(axis=1),
                          df[dates_7].values.sum(axis=1),
                          df[dates_8].values.sum(axis=1)]).T
    
    df["month_mean"] = month_sum.mean(axis=1)
    df["month_min"] = month_sum.min(axis=1)
    df["month_max"] = month_sum.max(axis=1)
    df["month_var"] = month_sum.var(axis=1)
    df["month_std"] = month_sum.std(axis=1)
    df["month_argmax"] = np.argmax(month_sum, axis=1)
    df["month_argmin"] = np.argmin(month_sum, axis=1)
    
    df["month_max_mean_diff"] = df["month_max"] - df["month_mean"]
    
    month_sum = pd.DataFrame(month_sum, columns=["month_4", "month_5", "month_6", "month_7", "month_8"])
    df = pd.concat([df, month_sum], axis=1)
    
    return df
    
train_df = get_agg(train_df)
test_df = get_agg(test_df)


train_df[agg_features]

,mean,max,var,std,argmax,max_mean_diff,argmin,month_mean,month_max,month_var,month_std,month_argmax,month_max_mean_diff,month_argmin,month_4,month_5,month_6,month_7,month_8
0,0.227016,0.754859,0.060181,0.245319,62,0.527843,6,3.178219,5.315207,2.089104,1.445373,3,2.136988,0,1.113779,3.229084,4.012807,5.315207,2.220217
1,0.270592,0.709037,0.064425,0.253821,51,0.438445,14,3.788287,5.509812,1.558655,1.248461,2,1.721525,0,2.255943,3.924852,5.509812,4.723104,2.527724
2,0.212570,0.741171,0.059005,0.242910,47,0.528601,2,2.975977,5.945200,4.057189,2.014247,2,2.969223,4,0.897648,2.908443,5.945200,4.409111,0.719480
3,0.270053,0.753284,0.061404,0.247798,35,0.483232,9,3.780737,6.923487,4.380717,2.093016,2,3.142751,0,1.250007,5.058194,6.923487,3.886033,1.785962
4,0.236884,0.833694,0.069019,0.262714,60,0.596809,8,3.316378,7.787268,6.416357,2.533053,3,4.470890,0,0.966005,2.454225,4.273642,7.787268,1.100749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,0.267562,0.801592,0.074936,0.273744,51,0.534030,2,3.745872,7.718678,4.795399,2.189840,3,3.972806,0,1.490575,3.032927,4.266962,7.718678,2.220218
4826,0.318033,0.845264,0.097682,0.312541,52,0.527231,6,4.452467,8.726955,7.223026,2.687569,3,4.274488,0,1.183015,3.805752,6.099797,8.726955,2.446818
4827,0.274709,0.844618,0.087591,0.295958,52,0.569909,4,3.845930,7.109506,4.393834,2.096147,3,3.263576,0,1.387849,2.798829,5.407300,7.109506,2.526166
4828,0.298225,0.767135,0.065390,0.255714,42,0.468910,7,4.175157,7.793224,5.237412,2.288539,2,3.618067,4,1.822635,5.242562,7.793224,4.369124,1.648239


# Адреса

In [22]:
from tqdm.notebook import tqdm
tqdm.pandas()

import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

locator = Nominatim(user_agent="myGeocoder")
train_df['location'] = train_df.progress_apply(lambda row: locator.reverse(f'{row.x}, {row.y}').raw, axis=1)
test_df['location'] = test_df.progress_apply(lambda row: locator.reverse(f'{row.x}, {row.y}').raw, axis=1)

  0%|          | 0/4830 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [14]:
adress_features = ["municipality", "municipality_type", "county", "county_type", "state", "state_type",
                   "ISO3166-2-lvl4", "region"]


def get_county_type(county):
    if "район" in county:
        return "район"
    if "округ" in county:
        return "округ"
    return "другое"
    
def get_state_type(state):
    if "область" in state:
        return "область"
    if "край" in state:
        return "край"
    return "другое"

def get_municipality_type(municipality):
    if "поселение" in municipality:
        return "поселение"
    if "сельсовет" in municipality:
        return "сельсовет"
    return "другое"

def get_adress(df):
    df['municipality'] = df["location"].apply(lambda x: x['address'].get('municipality', 'nan'))
    df['municipality_type'] = df["municipality"].apply(lambda county: get_municipality_type(county))
    df['county'] = df["location"].apply(lambda x: x['address'].get('county', 'nan'))
    df['county_type'] = df["county"].apply(lambda county: get_county_type(county))
    df['state'] = df["location"].apply(lambda x: x['address'].get('state', 'nan'))
    df['state_type'] = df["state"].apply(lambda state: get_state_type(state))
    df['ISO3166-2-lvl4'] = df["location"].apply(lambda x: x['address'].get('ISO3166-2-lvl4', 'nan'))
    df['region'] = df["location"].apply(lambda x: x['address'].get('region', 'nan'))
    df['country_code'] = df["location"].apply(lambda x: x['address'].get('country_code', 'nan'))
    
get_adress(train_df)
get_adress(test_df)

train_df[adress_features]

,municipality,municipality_type,county,county_type,state,state_type,ISO3166-2-lvl4,region
0,1,1,1,1,1,1,1,1
1,2,2,2,2,2,2,2,2
2,3,2,3,2,3,1,3,1
3,4,3,4,2,4,1,4,2
4,5,2,5,2,5,1,5,1
...,...,...,...,...,...,...,...,...
4825,330,2,102,2,1,1,1,1
4826,159,2,35,2,1,1,1,1
4827,1,1,197,1,3,1,3,1
4828,375,2,56,2,2,2,2,2


In [15]:
adress_agg_features = ["state_mean", "region_mean", "mean_state_mean_diff", "mean_region_mean_diff",
                       "municipality_mean", "mean_municipality_mean_diff"]
for adress_feature in ["municipality", "state", "region"]:
    val_dict = test_df.groupby(adress_feature)["mean"].mean().to_dict()
    train_df[f"{adress_feature}_mean"] = train_df[adress_feature].apply(lambda x: val_dict.get(x, 0.))
    train_df[f"mean_{adress_feature}_mean_diff"] = train_df[f"{adress_feature}_mean"] - df["mean"]
    test_df[f"{adress_feature}_mean"] = test_df[adress_feature].apply(lambda x: val_dict.get(x, 0.))
    test_df[f"mean_{adress_feature}_mean_diff"] = test_df[f"{adress_feature}_mean"] - sub["mean"]

train_df[adress_agg_features]

,state_mean,region_mean,mean_state_mean_diff,mean_region_mean_diff,municipality_mean,mean_municipality_mean_diff
0,0.282427,0.267558,0.055412,0.040542,0.256928,0.029912
1,0.260934,0.263769,-0.009658,-0.006823,0.293272,0.022680
2,0.244271,0.267558,0.031702,0.054988,0.264321,0.051751
3,0.259441,0.263769,-0.010611,-0.006284,0.000000,-0.270053
4,0.260780,0.267558,0.023896,0.030673,0.281373,0.044489
...,...,...,...,...,...,...
4825,0.282427,0.267558,0.014865,-0.000005,0.268338,0.000775
4826,0.282427,0.267558,-0.035606,-0.050476,0.289417,-0.028616
4827,0.244271,0.267558,-0.030438,-0.007152,0.256928,-0.017781
4828,0.260934,0.263769,-0.037291,-0.034457,0.000000,-0.298225


In [16]:
for adress_feature in adress_features:
    uniq_values = train_df[adress_feature].unique()
    uniq_values = {uniq_values[i]: i+1 for i in range(len(uniq_values))}
    train_df[adress_feature] = train_df[adress_feature].apply(lambda x: uniq_values.get(x, -1))
    test_df[adress_feature] = test_df[adress_feature].apply(lambda x: uniq_values.get(x, -1))
    
df[adress_features]

,municipality,municipality_type,county,county_type,state,state_type,ISO3166-2-lvl4,region
0,1,1,1,1,1,1,1,1
1,2,2,2,2,2,2,2,2
2,3,2,3,2,3,1,3,1
3,4,3,4,2,4,1,4,2
4,5,2,5,2,5,1,5,1
...,...,...,...,...,...,...,...,...
4825,330,2,102,2,1,1,1,1
4826,159,2,35,2,1,1,1,1
4827,1,1,197,1,3,1,3,1
4828,375,2,56,2,2,2,2,2


# Еще гео фичи

In [169]:
rot = ['rot_45_x', 'rot_45_y', 'rot_30_x', 'rot_30_y']

def rotation(data):
    data['rot_45_x'] = (0.707 * data['x']) + (0.707 * data['y'])
    data['rot_45_y'] = (0.707 * data['y']) + (0.707 * data['x'])
    data['rot_30_x'] = (0.866 * data['x']) + (0.5 * data['y'])
    data['rot_30_y'] = (0.866 * data['y']) + (0.5 * data['x'])
    
    
rotation(train_df)
rotation(test_df)

test_df[rot]

,rot_45_x,rot_45_y,rot_30_x,rot_30_y
0,66.397215,66.397215,65.774240,62.512316
1,75.559498,75.559498,74.154455,71.834619
2,64.930177,64.930177,65.623624,59.828458
3,71.168214,71.168214,70.803479,66.701161
4,61.998092,61.998092,63.209994,56.576985
...,...,...,...,...
4825,64.908712,64.908712,64.534519,60.876091
4826,64.915730,64.915730,64.869687,60.554482
4827,64.887449,64.887449,65.357923,60.011604
4828,76.712041,76.712041,74.448903,73.767006


# Почва

In [425]:
from tqdm.notebook import tqdm
tqdm.pandas()
import requests

x = 49.92074841855074
y = 39.60193178786137
def get_soil_prop(geo, x_c, y_c):
    if json.loads(geo).get("coordinates") is not None:
        coordinates = [[y_c, x_c]] + json.loads(geo)["coordinates"][0][:5]
    else:
        coordinates = [[y_c, x_c]]
       
    coordinates += [[y_c+0.001, x_c+0.001],
                    [y_c-0.001, x_c+0.001],
                    [y_c+0.001, x_c-0.001],
                    [y_c-0.001, x_c-0.001]]
    for coor in coordinates:
        x, y = coor[1], coor[0]
        req = f"https://rest.isric.org/soilgrids/v2.0/classification/query?lon={x}&lat={y}"
        res = requests.get(req)

        content = json.loads(res.content)
        
        if content.get('wrb_class_name') is not None:
            return content
    return content

train_df['soil_prop'] = train_df.progress_apply(lambda row: get_soil_prop(row['.geo'], row.x, row.y), axis=1)
test_df['soil_prop'] = test_df.progress_apply(lambda row: get_soil_prop(row['.geo'], row.x, row.y), axis=1)

test_df['soil_prop']

  0%|          | 0/4830 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



0       {'type': 'Point', 'coordinates': [52.384848092...
1       {'type': 'Point', 'coordinates': [52.064599645...
2       {'type': 'Point', 'coordinates': [49.944538241...
3       {'type': 'Point', 'coordinates': [52.550562564...
4       {'type': 'Point', 'coordinates': [55.372618656...
                              ...                        
2066    {'type': 'Point', 'coordinates': [54.015300223...
2067    {'type': 'Point', 'coordinates': [53.490585612...
2068    {'type': 'Point', 'coordinates': [53.137644371...
2069    {'type': 'Point', 'coordinates': [51.861706374...
2070    {'type': 'Point', 'coordinates': [53.790726783...
Name: soil_prop, Length: 2071, dtype: object

In [435]:
soil_features = ['wrb_class_name', 'wrb_class_value']
wrb_class_probability = ['None', 'Regosols','Phaeozems','Chernozems','Kastanozems','Gypsisols','Albeluvisols',
                         'Calcisols','Cambisols','Solonetz', 'Solonchaks','Fluvisols','Luvisols','Cryosols', 
                         'Leptosols','Histosols','Gleysols','Podzols','Vertisols','Arenosols']

df['wrb_class_name'] = df['soil_prop'].apply(lambda x: str(x['wrb_class_name']))
df['wrb_class_value'] = df['soil_prop'].apply(lambda x: x['wrb_class_value'])
df['soil_prop_probs'] = df['soil_prop'].apply(lambda x: {e[0]: e[1] for e in x['wrb_class_probability']})

sub['wrb_class_name'] = sub['soil_prop'].apply(lambda x: str(x['wrb_class_name']))
sub['wrb_class_value'] = sub['soil_prop'].apply(lambda x: x['wrb_class_value'])
sub['soil_prop_probs'] = sub['soil_prop'].apply(lambda x: {e[0]: e[1] for e in x['wrb_class_probability']})

soil_features += wrb_class_probability

for wrb_class in wrb_class_probability:
    train_df[wrb_class] = train_df['soil_prop_probs'].apply(lambda x: x.get(wrb_class, -1))
    test_df[wrb_class] = test_df['soil_prop_probs'].apply(lambda x: x.get(wrb_class, -1))

train_df.fillna(-1, inplace=True)
test_df.fillna(-1, inplace=True)
for col in ['wrb_class_name']:
    uniq_values = df[col].unique()
    uniq_values = {uniq_values[i]: i+1 for i in range(len(uniq_values))}
    train_df[col] = train_df[col].apply(lambda x: uniq_values.get(x, -1))
    test_df[col] = test_df[col].apply(lambda x: uniq_values.get(x, -1))
    
test_df[soil_features]

,wrb_class_name,wrb_class_value,None,Regosols,Phaeozems,Chernozems,Kastanozems,Gypsisols,Albeluvisols,Calcisols,...,Solonchaks,Fluvisols,Luvisols,Cryosols,Leptosols,Histosols,Gleysols,Podzols,Vertisols,Arenosols
0,1,-1.0,-1,-1,-1,-1,-1,-1,-1.0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0
1,2,7.0,-1,-1,-1,31,18,-1,-1.0,-1,...,7,-1,11,-1,-1,-1,6,-1,-1,-1.0
2,3,5.0,-1,9,-1,-1,-1,-1,-1.0,16,...,15,12,-1,-1,-1,-1,-1,-1,-1,8.0
3,4,24.0,-1,24,-1,-1,-1,-1,-1.0,7,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,8.0
4,5,13.0,-1,23,-1,-1,-1,24,-1.0,-1,...,19,13,-1,-1,-1,-1,-1,-1,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,1,-1.0,-1,-1,-1,-1,-1,-1,-1.0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0
4826,1,-1.0,-1,-1,-1,-1,-1,-1,-1.0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0
4827,1,-1.0,-1,-1,-1,-1,-1,-1,-1.0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1.0
4828,2,7.0,-1,-1,17,24,-1,-1,-1.0,-1,...,-1,-1,-1,-1,9,-1,8,-1,-1,-1.0


In [455]:
x = 56.413192
y = 42.500831
def get_soil_prop_v2(geo, x_c, y_c):
    if json.loads(geo).get("coordinates") is not None:
        coordinates = [[y_c, x_c]] + json.loads(geo)["coordinates"][0][:1]
    else:
        coordinates = [[y_c, x_c]]
       
    coordinates += [[y_c+0.001, x_c+0.001],
                    [y_c-0.001, x_c+0.001]]
        
    for coor in coordinates:
        x, y = coor[1], coor[0]
        req = f"https://rest.isric.org/soilgrids/v2.0/properties/query?lon={x}&lat={y}"
        res = requests.get(req)

        content = json.loads(res.content)
        if content.get("properties"):
            content = content['properties']
            if content.get('layers'):
                content = content['layers']
                if content[0].get('depths'):
                    if content[0]['depths'][0].get('values'):
                        if content[0]['depths'][0]["values"]["mean"] is not None:
                            return content
    return content

train_df['properties_layers'] = train_df.progress_apply(lambda row: get_soil_prop_v2(row[".geo"], row.x, row.y), axis=1)
test_df['properties_layers'] = test_df.progress_apply(lambda row: get_soil_prop_v2(row[".geo"], row.x, row.y), axis=1)

  0%|          | 0/2071 [00:00<?, ?it/s]

In [456]:
all_types = ["bdod", "cec", "cfvo", "clay", "nitrogen", "ocd", "ocs", "phh2o", "sand", "silt", "soc"]

def get_mean(d, j):
    try:
        return d[j]['values']['mean']
    except Exception:
        return -1
    
def get_all_soil_features(df):
    for i, type_ in enumerate(all_types):
        for j in range(3):
            df[f'{type_}_{j}'] = df['properties_layers'].apply(lambda x: get_mean(x[i]['depths'], j))
        
get_all_soil_features(df)
get_all_soil_features(sub)

all_soil_features = []
for i in range(3):
    all_soil_features += [f'{type_}_{i}' for type_ in all_types]
    
train_df.fillna(-1, inplace=True)
test_df.fillna(-1, inplace=True)

train_df[all_soil_features]

,bdod_0,cec_0,cfvo_0,clay_0,nitrogen_0,ocd_0,ocs_0,phh2o_0,sand_0,silt_0,...,cec_2,cfvo_2,clay_2,nitrogen_2,ocd_2,ocs_2,phh2o_2,sand_2,silt_2,soc_2
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0
1,111.0,240.0,131.0,317.0,604.0,449.0,64.0,71.0,315.0,369.0,...,260.0,134.0,320.0,312.0,252.0,-1,74.0,317.0,363.0,265.0
2,135.0,234.0,160.0,298.0,261.0,308.0,33.0,76.0,237.0,464.0,...,213.0,141.0,299.0,212.0,225.0,-1,77.0,236.0,466.0,173.0
3,135.0,237.0,60.0,246.0,330.0,245.0,38.0,78.0,276.0,478.0,...,235.0,60.0,256.0,168.0,162.0,-1,78.0,272.0,471.0,196.0
4,147.0,186.0,111.0,241.0,97.0,146.0,19.0,74.0,420.0,340.0,...,180.0,114.0,240.0,61.0,92.0,-1,75.0,419.0,342.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0
4826,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0
4827,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0
4828,102.0,397.0,85.0,268.0,540.0,518.0,70.0,66.0,349.0,382.0,...,349.0,100.0,303.0,217.0,256.0,-1,67.0,340.0,357.0,279.0


# Погода

In [ ]:
from meteostat import Point, Daily, Stations
import meteostat
from datetime import datetime


weather_cols = ["tavg", "tmin", "tmax", "prcp", "snow", "wdir", "wspd", "wpgt", "pres", "tsun"]
def get_weather_info(df):
    for date_col in tqdm.tqdm(date_cols):
        weather_dict = {wc: [] for wc in weather_cols}
        for i, row in df.iterrows():
            date = date_col.split('-')
            y, m, d = int(date[0]), int(date[1]), int(date[2])
            start = datetime(y, m, d)
            end = datetime(y, m, d)
            
            location = Point(row.x, row.y)
            
            stations = Stations()
            stations = stations.nearby(row.x, row.y)
            station = stations.fetch(10)

            data = Daily(station, start, end)
            data = data.fetch()
            data.fillna(0., inplace=True)
            
            for wc in weather_cols:
                try:
                    weather_dict[wc].append(np.median(data[wc].values))
                except Exception as e:
                    print(e, data)
                    weather_dict[wc].append(0.)
                
        for wc in weather_cols:
            df[f"{date_col}_{wc}"] = weather_dict[wc]

            
weather_date_cols = []
for date_col in date_cols[:10]:
    for wc in weather_cols:
        weather_date_cols.append(f"{date_col}_{wc}")

get_weather_info(train_df)
get_weather_info(test_df)

weather_date_cols = [wc for wc in weather_date_cols if (df[wc] > 0).sum() > 4000]
train_df[weather_date_cols]

# Финальный список фичей

In [5]:
final_features = ['2021-05-21', 'month_mean', '2021-04-25_tavg', 'argmin', '2021-04-25_tmax', '2021-04-23_wdir', '2021-04-30',
                '2021-04-28', '2021-04-18_wdir', '2021-06-27', 'Fluvisols', 'month_var', 'mean_state_mean_diff', 'max', 'area',
                '2021-06-25', '2021-06-22', 'month_4', 'month_5', '2021-04-19_tavg', '2021-04-19_wdir', 'Arenosols', 'month_7',
                'Regosols', 'Cryosols', 'municipality', 'rot_45_y', '2021-04-25_pres', 'Luvisols', 'y_n_near_01', '2021-04-26_wdir',
                'Cambisols', '2021-06-18', '2021-06-12', '2021-04-29', '2021-05-29', '2021-04-15_wdir', '2021-07-15', '2021-08-01',
                'wrb_class_name', 'municipality_type', 'bdod_1', '2021-04-22', '2021-05-15', '2021-04-16_wdir', '2021-04-16',
                '2021-04-22_pres', '2021-06-04', 'Chernozems', 'bdod_0', '2021-06-13', '2021-07-07', 'municipality_mean', '2021-06-05',
                'sand_1', '2021-04-27_wspd', 'rot_30_y', 'month_8', '2021-07-05', '2021-05-09', '2021-07-08', 'crop_3',
                'argmax', '2021-06-19', '2021-08-10', '2021-04-15', 'rot_30_x', '2021-05-27', '2021-04-15_tavg', 'y',
                '2021-04-15_wspd', 'Leptosols', '2021-04-26', 'Histosols', 'cfvo_0', '2021-06-16', '2021-04-23_tavg',
                '2021-04-22_wspd', '2021-04-26_tmax', '2021-05-03', 'Albeluvisols', 'silt_0', '2021-05-24', '2021-06-06', 'Gleysols',
                '2021-04-27_pres', '2021-07-20', '2021-04-18_tmin', '2021-04-25_wspd', 'std', '2021-05-02', '2021-05-01', '2021-08-07',
                'crop_1', 'silt_2', '2021-04-23_pres', 'month_argmin', '2021-05-17', '2021-04-18_tmax', '2021-04-27', '2021-06-09',
                '2021-04-15_pres', 'cec_1', '2021-04-16_tmin', '2021-07-04', 'mean', 'mean_municipality_mean_diff', '2021-04-26_wspd',
                'Kastanozems', 'x', '2021-04-16_tavg', 'phh2o_1', 'ocs_2', 'crop_5', '2021-05-26', 'nitrogen_0', 'rot_45_x',
                '2021-04-15_tmax', 'clay_1', '2021-04-23_tmax', '2021-04-19_wspd', '2021-06-10', 'state', '2021-04-16_wspd',
                '2021-06-07', 'Gypsisols', '2021-04-18', '2021-06-02', '2021-07-26', 'var', 'ISO3166-2-lvl4', 'nitrogen_1', 'cec_2',
                '2021-05-07', '2021-04-27_tavg', 'Solonchaks', 'Phaeozems', 'clay_0', '2021-04-26_pres', 'y_n_near_01_on_05',
                '2021-04-20_tmax', '2021-04-23_tmin', '2021-05-10', 'state_type', 'phh2o_0', '2021-04-27_wdir', 'region_mean',
                'nitrogen_2', '2021-07-29', '2021-04-19', 'soc_1', 'clay_2', '2021-04-16_pres', 'state_mean', '2021-08-23',
                '2021-04-18_tavg', 'ocd_1', 'ocs_1', '2021-04-20_pres', '2021-07-09', 'ocd_0', 'ocd_2', 'month_std', 'mean_region_mean_diff',
                'Vertisols', '2021-07-13', '2021-05-08', 'month_max_mean_diff', 'x_n_near_01', 'wrb_class_value', '2021-04-22_tmin', '2021-08-11',
                '2021-06-20', '2021-05-16', '2021-04-20_wdir', '2021-04-22_tmax', 'max_mean_diff', 'None', '2021-04-19_tmax', '2021-04-23_wspd', 
                'bdod_2', '2021-04-18_wspd', '2021-04-20_tavg', '2021-06-03', 'crop_6', 'Podzols', 'cec_0', 'silt_1', 'county_type', 'month_max',
                '2021-06-28', '2021-07-31', 'x_n_near_01_on_05', '2021-07-17', 'soc_2', '2021-04-22_prcp', 'crop_0', '2021-08-13',
                '2021-05-04', '2021-07-27', '2021-04-18_pres', 'cfvo_2', 'sand_0', 'cfvo_1', 'x_n_near_05', '2021-04-16_tmax', 
                'phh2o_2', '2021-04-25', 'county', '2021-08-27', '2021-04-27_tmax', '2021-08-12', '2021-04-20_wspd', '2021-04-19_pres',
                'Solonetz', 'sand_2', 'y_n_near_05', 'crop_2', '2021-04-23', 'region', '2021-04-15_tmin', '2021-04-22_tavg',
                'month_argmax', '2021-04-26_tavg', '2021-05-19', '2021-04-20', 'Calcisols', '2021-05-20', '2021-04-22_wdir',
                '2021-04-25_wdir', 'ocs_0', 'month_6', 'soc_0', 'crop_4']


# Сохраняем датасет

In [ ]:
train_df[final_features + ['crop']].to_csv("train.csv", index=False)
test_df[final_features].to_csv("test.csv", index=False)